In [1]:
import rasterio

In [2]:
ds_hand = rasterio.open("./data/080102hand.tif")

In [3]:
ds_hand.crs

CRS.from_epsg(4269)

In [4]:
ds_sar = rasterio.open("./data/16SCF_0_4-MASK.tiff")

In [5]:
ds_sar.crs

CRS.from_epsg(32616)

In [6]:
ds_sar.shape

(2004, 2004)

In [7]:
from rasterio.enums import Resampling
from rasterio.vrt import WarpedVRT

vrt = WarpedVRT(ds_hand, crs=ds_sar.crs, resampling=Resampling.bilinear)

In [8]:
source_window = rasterio.windows.from_bounds(*ds_sar.bounds, transform=vrt.transform).round_offsets()
target_window = rasterio.windows.Window(source_window.col_off, source_window.row_off, ds_sar.width, ds_sar.height)
data = vrt.read(1, window=target_window)

In [9]:
data.shape

(2004, 2004)

In [10]:
data

array([[ 6.31660469,  5.96711612,  5.58032625, ..., 13.40665376,
        13.50962887, 13.62637697],
       [ 6.19250942,  5.87042185,  5.4508645 , ..., 13.26148488,
        13.34436455, 13.45968823],
       [ 6.74726091,  6.41029673,  5.99744877, ..., 12.68845637,
        12.84526903, 13.17922707],
       ...,
       [12.89218721, 13.23017384, 13.34177359, ...,  5.36011372,
         4.70496055,  3.90625704],
       [12.67636839, 13.06880402, 13.31755496, ...,  5.13609158,
         4.43340149,  3.73841828],
       [12.11516606, 12.67585582, 13.16802635, ...,  5.03621488,
         4.29526243,  3.47407371]])

In [11]:
dst = rasterio.open("./data/output.tif", "w", 
                    driver="GTiff", 
                    width=data.shape[1], 
                    height=data.shape[0], 
                    count=1, 
                    crs=vrt.crs, 
                    transform=ds_sar.transform, 
                    dtype=data.dtype)
dst.write(data, indexes=1)